In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from gensim.models.doc2vec import Doc2Vec

caminho_modelo = 'modelos/doc2vec.text_0_100_10'

In [2]:
# Carregamento do dataset.
df = pd.read_csv('../dados/articles_limpo.csv', nrows=5000)
df.columns

Index(['title', 'text', 'date', 'category', 'link'], dtype='object')

In [ ]:
df.to_csv('../dados/articles_limpo.csv', index=False)

In [ ]:
# Carregando o modelo doc2vec (títulos).
modelo = Doc2Vec.load(caminho_modelo)

In [ ]:
from classificadores import Classificador

c =  Classificador()

caminho_df = '../dados/articles_limpo.csv'
df = pd.read_csv(caminho_df)
classif, s_classif = c.pre_proc(df)
#treino, teste = c.dataset_treino(classif, s_classif, modelo)
c.dataset_treino(classif, s_classif, modelo)
clf = LogisticRegression(random_state=0, max_iter=150, solver='sag')#, C=0.8)


In [ ]:
# Treinando o modelo.
"""
vetores = list()
classes = list()
for index, row in treino.iterrows():
    vetores.append(row['vetor'])
    classes.append(row['classe'])
"""

#clf.fit(vetores, classes)
clf.fit(list(classif['vetor']), list(classif['classe']))

In [ ]:
# Classificando as novas amostras.
vetores = list()
for index, row in s_classif.head().iterrows():
    vetores.append(row['vetor'])
    print(clf.predict([row['vetor']]))
#classificados = clf.predict(vetores)

In [ ]:
list(classif['vetor'])

In [ ]:
# Treinando o Naive Bayes.
gnb = GaussianNB()
naive = gnb.fit(vetores_treino[:69854], classes_treino[:69854])
# 29937
#for vetor, classe in zip(vetores_treino[29937:], classes_treino[29937:]):
#    naive.predict([vetor, classe])
acc = naive.score(vetores_treino[29937:], classes_treino[29937:])
acc

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(vetores_treino[:69854], classes_treino[:69854])
clf.score(vetores_treino[29000:], classes_treino[29000:])

In [ ]:

###
### A partir desta célula a implementação refere-se ao algoritmo TD-IDF
###

In [19]:
# Limpando o texto.
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

df = df[df['text'].notna()]

categorias_finais = ['tec', 'esporte', 'ilustrada', 'mercado', 'poder', 'mundo']
categorias_diluidas = list(set(df['category']) - set(categorias_finais))

# Agrupando coluna de tecnologia com ciência.
# df["category"]= df["category"].replace("ciencia", "tec")

# Criando dataset com categorias finais para teste.
docs_classif = df
for cat in categorias_diluidas:
    docs_classif = docs_classif[docs_classif['category'] != cat]

tokenizer = RegexpTokenizer(r'\w+')
documentos = list()
stpw = {}
for p in stopwords.words("portuguese"):
    stpw[p] = True

for row in docs_classif.itertuples():
    doc = list()
    for p in tokenizer.tokenize(row.text.lower()):
        if p not in stpw:
            doc.append(p)
    documentos.append(' '.join(doc))

docs_classif['texto_limpo'] = documentos
df_treino = docs_classif[['texto_limpo', 'category']]

In [34]:
## Aplicando o tf-idf
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

#instantiate CountVectorizer()
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(df_treino['texto_limpo'])

tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [43]:
# Aplicando os algoritimos.
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

alg = 4

# Regressão Logistica.
if alg == 1:
    clf = LogisticRegression(random_state=0, max_iter=150, solver='sag')
# Naive Bayes.
elif alg == 2:
    clf = GaussianNB()
# Rede Neural.
elif alg == 3:
    clf = MLPClassifier(solver='adam', hidden_layer_sizes=(100), learning_rate_init=0.002, \
            learning_rate= 'adaptive', max_iter=200)
else:
    clf = RandomForestClassifier(max_depth=2, random_state=0)

scores = cross_val_score(clf, X_train_counts, df_treino['category'], cv=5, scoring='f1_macro', n_jobs=-1)
print("F1-Measure: ", scores)


F1-Measure:  [0.26915906 0.30795149 0.29875718 0.26647858 0.28627336]
